In [18]:
import os
import pandas as pd

def process_game_data(main_folder):
    # List of required files
    required_files = [
        'overview.csv', 'copies sold.csv', 'revenue.csv', 'top seller rank.csv',
        'ccu.csv', 'price.csv', 'review.csv', 'followers.csv',
        'average playtime.csv', 'outstanding wishlist.csv', 'positive reviews.csv'
    ]
    
    # Initialize a DataFrame to store game info
    df_info = pd.DataFrame(columns=['game_name', 'location'])

    # Iterate through each subfolder in the main folder
    for game_folder in os.listdir(main_folder):
        game_path = os.path.join(main_folder, game_folder)
        if os.path.isdir(game_path):
            # Check if all required files are present
            if all(os.path.isfile(os.path.join(game_path, file)) for file in required_files):
                # Load each file into a DataFrame
                df_overview = pd.read_csv(os.path.join(game_path, 'overview.csv'))
                df_copies_sold = pd.read_csv(os.path.join(game_path, 'copies sold.csv'))
                df_revenue = pd.read_csv(os.path.join(game_path, 'revenue.csv'))
                df_top_seller_rank = pd.read_csv(os.path.join(game_path, 'top seller rank.csv'))
                df_ccu = pd.read_csv(os.path.join(game_path, 'ccu.csv'))
                df_price = pd.read_csv(os.path.join(game_path, 'price.csv'))
                df_reviews = pd.read_csv(os.path.join(game_path, 'review.csv'))
                df_followers = pd.read_csv(os.path.join(game_path, 'followers.csv'))
                df_average_playtime = pd.read_csv(os.path.join(game_path, 'average playtime.csv'))
                df_outstanding_wishlist = pd.read_csv(os.path.join(game_path, 'outstanding wishlist.csv'))
                df_positive_reviews = pd.read_csv(os.path.join(game_path, 'positive reviews.csv'))

                # Process df_price based on the number of columns
                if len(df_price.columns) > 2:
                    df_price = df_price.T
                    df_price = df_price[1:]
                    df_price = df_price.reset_index()
                    df_price = df_price.rename(columns={'index': 'date'})
                    df_price['date'] = df_price['date'].str[4:]
                    df_price = df_price.rename(columns={0: 'price'})
                else:
                    df_price = df_price.rename(columns={'x': 'date', 'y': 'price'})
                    df_price['date'] = pd.to_datetime(df_price['date'], unit='ms')
                    filled_rows = []
                    for i in range(len(df_price) - 1):
                        current_price = df_price.loc[i, 'price']
                        current_date = df_price.loc[i, 'date']
                        next_date = df_price.loc[i + 1, 'date']
                        date_range = pd.date_range(start=current_date, end=next_date, freq='D')
                        for date in date_range:
                            filled_rows.append({'date': date, 'price': current_price})
                    filled_rows.append({'date': df_price.loc[len(df_price) - 1, 'date'], 'price': df_price.loc[len(df_price) - 1, 'price']})
                    df_price = pd.DataFrame(filled_rows)
                    df_price.reset_index(drop=True, inplace=True)
                    df_price['date'] = df_price['date'].dt.normalize()
                    df_price = df_price.drop_duplicates(subset='date', keep='first').reset_index(drop=True)
                    df_price = df_price.drop(df_price.index[-1])

                # Process other DataFrames
                def process_df(df, column_name):
                    df = df.T
                    df = df[1:]
                    df = df.reset_index()
                    df = df.rename(columns={'index': 'date'})
                    df['date'] = df['date'].str[4:]
                    df = df.rename(columns={0: column_name})
                    return df

                df_copies_sold = process_df(df_copies_sold, 'copies sold')
                df_revenue = process_df(df_revenue, 'revenue')
                df_top_seller_rank = process_df(df_top_seller_rank, 'top seller rank')
                df_ccu = process_df(df_ccu, 'ccu')
                df_reviews = process_df(df_reviews, 'reviews')
                df_followers = process_df(df_followers, 'followers')
                df_average_playtime = process_df(df_average_playtime, 'average playtime')
                df_outstanding_wishlist = process_df(df_outstanding_wishlist, 'outstanding wishlist')
                df_positive_reviews = process_df(df_positive_reviews, 'positive reviews')

                # Process df_overview
                tag_categories = {
                    'tag game type': [
                        'Action Roguelike', '2D Platformer', 'Hack and Slash', 'Roguelite', 'Action RPG', 'FPS', 'Multiplayer',
                        'Open World', 'Survival', 'Crafting', 'Co-op Campaign', 'Base Building', 'Simulation', 'Exploration',
                        'Visual Novel', 'Shooter', 'RPG', 'Strategy', 'Sandbox', 'Action-Adventure', 'Sports', 'Puzzle', 'Fighting',
                        'Roguelike', 'Adventure', 'Card Game', 'RTS'
                    ],
                    'tag style': [
                        'Souls-like', 'Cyberpunk', 'Story Rich', 'Multiple Endings', 'Anime', 'Pixel Graphics', 'Retro', 'Cartoony',
                        'Fantasy', 'Dark Fantasy', 'Comedy', 'LGBTQ+', 'Realistic', 'Gothic', 'Mythology', 'Noir', 'Dystopian',
                        'Sci-fi', 'Lovecraftian', 'Atmospheric', 'Cute'
                    ],
                    'tag mechanics': [
                        'Side Scroller', 'Controller', 'Choices Matter', 'Crafting', 'Base Building', 'Co-op Campaign',
                        'Turn-Based', 'Real-Time', 'Procedural Generation', 'Deckbuilding', 'Inventory Management',
                        'Physics', 'Roguelike Deckbuilder', 'Tactical', 'PvP', 'PvE', 'Resource Management', 'Exploration', 'Combat',
                        'Building', 'Moddable', 'Level Editor', 'Narration'
                    ],
                    'tag perspective': [
                        '2D', '3D', 'Isometric', 'First-Person', 'Third Person', 'Top-Down', 'Side-Scrolling', 'Third-Person Shooter',
                        'VR', 'Over-the-Shoulder', '3D Vision', '2.5D', 'First-Person Shooter'
                    ]
                }

                def categorize_tags(tag_string, categories):
                    tags = tag_string.split(',')
                    categorized = {key: [] for key in categories}
                    for tag in tags:
                        tag = tag.strip()
                        found = False
                        for category, keywords in categories.items():
                            if tag in keywords:
                                categorized[category].append(tag)
                                found = True
                        if not found:
                            continue
                    for category in categorized:
                        if not categorized[category]:
                            categorized[category] = ['-']
                    return categorized

                for category in tag_categories.keys():
                    df_overview[category] = df_overview['tags'].apply(lambda x: ', '.join(categorize_tags(x, tag_categories)[category]) if pd.notnull(x) else '-')
                df_overview = df_overview.drop(columns=['tags'])

                df_overview = df_overview.drop(columns=['languages', 'price', 'features', 'early access exit date', 'description', 'first release date'])
                genres_split = df_overview['genres'].str.split(',', expand=True)
                df_overview = df_overview.assign(
                    genre_1=genres_split.get(0, '-'),
                    genre_2=genres_split.get(1, '-'),
                    genre_3=genres_split.get(2, '-')
                )
                cols = df_overview.columns.tolist()
                dev_idx = cols.index('developers')
                new_cols_order = cols[:dev_idx+1] + ['genre_1', 'genre_2', 'genre_3'] + cols[dev_idx+1:]
                df_overview = df_overview[new_cols_order]
                df_overview = df_overview.drop(columns=['genres'])

                df_overview_repeated = pd.concat([df_overview]*len(df_price), ignore_index=True)
                df_game = pd.concat([df_overview_repeated.reset_index(drop=True), df_price.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_copies_sold.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_revenue.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_top_seller_rank.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_ccu.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_reviews.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_followers.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_average_playtime.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_outstanding_wishlist.reset_index(drop=True)], axis=1)
                df_game = pd.concat([df_game, df_positive_reviews.reset_index(drop=True)], axis=1)
                df_game = df_game.loc[:, ~df_game.columns.duplicated(keep='first')]

                is_free_game = df_price['price'].eq(0).all()
                df_game['free game'] = is_free_game
                if not is_free_game:
                    non_zero_price_index = df_price[df_price['price'] != 0].index
                    df_game = df_game.loc[non_zero_price_index].reset_index(drop=True)
                    df_price = df_price.loc[non_zero_price_index].reset_index(drop=True)

                df_game['daily copies sold'] = df_game['copies sold'].diff().fillna(0)
                df_game['daily revenue'] = df_game['revenue'].diff().fillna(0)
                df_game['daily reviews'] = df_game['reviews'].diff().fillna(0)
                df_game['daily outstanding wishlist'] = df_game['outstanding wishlist'].diff().fillna(0)

                gameinfo_path = os.path.join(game_path, 'gameinfo.csv')
                df_game.to_csv(gameinfo_path, index=False)
                print(f"Game info for {game_folder} saved at {gameinfo_path}")

                df_info = pd.concat([df_info, pd.DataFrame({'game_name': [game_folder], 'location': [gameinfo_path]})], ignore_index=True)

    df_info.to_csv(os.path.join(main_folder, 'game_info_summary.csv'), index=False)

    # Combine all gameinfo.csv files into a massive DataFrame
    df_steamgame = pd.DataFrame()
    for game_folder in os.listdir(main_folder):
        game_path = os.path.join(main_folder, game_folder)
        gameinfo_path = os.path.join(game_path, 'gameinfo.csv')
        if os.path.isfile(gameinfo_path):
            df_game = pd.read_csv(gameinfo_path)
            df_steamgame = pd.concat([df_steamgame, df_game], ignore_index=True)

    df_steamgame.to_csv(os.path.join(main_folder, 'df_steamgame.csv'), index=False)
    print("ALL TASKS DONE!!!!!!")

# Example usage
main_folder = '../data/raw/game lib'
process_game_data(main_folder)

Game info for Abiotic Factor saved at ../data/raw/game lib\Abiotic Factor\gameinfo.csv
Game info for Ace Attorney Investigations Collection saved at ../data/raw/game lib\Ace Attorney Investigations Collection\gameinfo.csv
Game info for Age of Mythology Retold saved at ../data/raw/game lib\Age of Mythology Retold\gameinfo.csv
Game info for Alone in the Dark saved at ../data/raw/game lib\Alone in the Dark\gameinfo.csv
Game info for ANIMAL WELL saved at ../data/raw/game lib\ANIMAL WELL\gameinfo.csv
Game info for Anomaly Agent saved at ../data/raw/game lib\Anomaly Agent\gameinfo.csv
Game info for Another Crabs Treasure saved at ../data/raw/game lib\Another Crabs Treasure\gameinfo.csv
Game info for Apollo Justice Ace Attorney Trilogy saved at ../data/raw/game lib\Apollo Justice Ace Attorney Trilogy\gameinfo.csv
Game info for ASKA saved at ../data/raw/game lib\ASKA\gameinfo.csv
Game info for ASTRA Knights of Veda saved at ../data/raw/game lib\ASTRA Knights of Veda\gameinfo.csv
Game info for 